# IFT712 - Projet de session

* 6 méthodes de classification 
* Analyse des données
* Recherche d'hyperparamètres
* Visualisation des résultats

# Configuration

In [ ]:
# Vérifier si les bibliothèques nécessaires sont déjà installées
try:
    import pandas as pd
    import numpy as np
    import sklearn
    import matplotlib
    import seaborn
    import os
    import warnings
    import plotly.express as px

    print("Les bibliothèques sont déjà installées.")
except ImportError:
    # Installer les bibliothèques manquantes
    %pip install pandas numpy scikit-learn matplotlib seaborn os warnings plotly.express

    # Redémarrer le noyau du notebook après l'installation (optionnel)
    # Pour effectuer le redémarrage automatique, décommentez la ligne suivante
    # import IPython
    # IPython.Application.instance().kernel.do_shutdown(True)

In [54]:
'''
    Imporation des bibliothèques python générales
'''
import numpy as np
import itertools
from sklearn.datasets import make_classification
import os
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import LabelEncoder

'''
    Imporation des bibliothèques spécifiques au projet
'''
import utils
from linear_classifier import LinearClassifier
from two_layer_classifier import TwoLayerClassifier

"""
    Préludes
"""
warnings.filterwarnings("ignore")
pd.set_option("display.max_rows",None)
matplotlib.style.use('ggplot')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Données du projet : la classification linéaire ...

https://www.kaggle.com/datasets/rashikrahmanpritom/heart-attack-analysis-prediction-dataset?select=heart.csv

In [ ]:
# Importer les données, visualiser le dataframe, expliquer le contenu des variables etc.
# Voir class DataManager():

Séparer le jeu de données en trois parties: l'ensemble **d'entraînement**, de **validation** et de **test** (_train_, _val_ et _test_).

In [ ]:
# CODE 
# Voir class DataManager():

# Analyse exploratoire des données

### Matrice de correlation 

### Distribution de quelques variables par rapport à la cible
* On observe les distibution d'une variable suivant les classes cibles. Par exemple : la distritution de la variable "sex" par rapport à la cible "HeartDisease"

### Distribution de toutes les variables 

### Boite à moustache (Boxplot)
* Aide à voir les valeurs statistiques comme la moyenne, les quartiles, et surtout les valeurs abérrantes

# Nettoyage des données

* Gestion des valeurs NaN ou valeurs manquantes

* Mise à l'échelle des variables

* Gestion des variables catégorielles

### Gestion des NaN ou valeurs manquantes

In [ ]:
# Utiliser DataProcessor > scale_features()

### Mise à l'échelle des variables

In [ ]:
# Utiliser DataProcessor > andle_missing_values()

### Gestion des variables catégorielles

In [ ]:
# Utiliser DataProcessor > handle_categorical_variables()

# Entrainer les données sur les 6 algorithmes
* Algorithmes choisis : 
    - Random Forest Classifier,
    - AdaBoost Classifier, 
    - Logistic Regression, 
    - MLP Classifier
    - SVM
    - Linear Discriminant Analysis
* Séparer les données en esembles de d'entraienement, de validation et de test.
* Faire la validation croisée stratifiée. 
* Implémenter la recherche d'hyperparamètres
* Visualiser les courbes d'apprentissage et de validation

### Rechercher de meilleurs hyperparamètres


# Vérifier la généralisation sur l'ensemble de test avec les meilleurs huperparamètres

# Tracer les frontières de décision (si possible)
* On pourra sélectioner 2 ou 3 variables les plus importantes pour la classification

